# <center> Variational Post-selection
## <center> H4 (square)

Since `PyScf` is not portable in Windows, we have to calculate the following codebox in `Linux`. The MolecularData file will be saved in `filename.hdf5`. Use command `cp filename.hdf5 /mnt/d` in WSL to copy it to your Windows `D:`.

In [ ]:
# in Linux
from openfermion.chem import MolecularData
from openfermionpyscf import run_pyscf

d = 1.7
geometry = [('H', (0, 0, 0)), ('H', (0, 0, d)), ('H', (0, d, d)), ('H', (0, d, 0))]
basis = "sto-3g"
multiplicity = 1
description = "h4_square"
molecule = MolecularData(geometry, basis, multiplicity, description=description, filename="h4_square_17")
molecule = run_pyscf(molecule, run_mp2=True, run_cisd=True, run_ccsd=True, run_fci=True)
molecule.save()


Now, we can further process in Windows. (Don't forget to move the MolecularData file to the file where your `.ipynb` is.)

## Setup

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorcircuit as tc
import cotengra
import quimb

optc = cotengra.ReusableHyperOptimizer(
    methods=["greedy"],
    parallel="ray",
    minimize="combo",
    max_time=30,
    max_repeats=1024,
    progbar=True,
)
tc.set_contractor("custom", optimizer=optc, preprocessing=True)

K = tc.set_backend("tensorflow")
tc.set_dtype("complex128")

from openfermion.chem import MolecularData
from openfermion.linalg import LinearQubitOperator
from openfermion.transforms import (
    get_fermion_operator,
    jordan_wigner
)

## Load Molecule

In [ ]:
# check the informations below
d = 1.7
geometry = [('H', (0, 0, 0)), ('H', (0, 0, d)), ('H', (0, d, d)), ('H', (0, d, 0))]
basis = "sto-3g"
multiplicity = 1
description = "h4_square"
molecule = MolecularData(geometry, basis, multiplicity, description=description, filename="models/h4_square_17")   # check the filename
molecule.load()

In [ ]:
# Get Fermionic Hamiltonian
mh = molecule.get_molecular_hamiltonian()
fh = get_fermion_operator(mh)

# Transform into qubit Hamiltonian
b = jordan_wigner(fh) 
ns = LinearQubitOperator(b).n_qubits
print("system qubit number:", ns)

# Transform from OpenFermion to TensorCircuit
lsb, wb = tc.templates.chems.get_ps(b, ns)
mb = tc.quantum.PauliStringSum2COO_numpy(lsb, wb)
mbd = mb.todense()
mbd_tf = tc.array_to_tensor(mbd)

In [ ]:
e_anal = np.min(quimb.eigvalsh(mbd))
print("fci energy: {:.10f} \nccsd energy: {:.10f} \nhf energy: {:.10f} \nanalytical energy: {:.10f} \n".format(molecule.fci_energy, molecule.ccsd_energy, molecule.hf_energy, e_anal))

## <center> Conventional VQE

In [ ]:
def U_conventional(ns, depth, params):
    params = K.cast(params, "complex128")
    c = tc.Circuit(ns)
    idx = 0
    
    c.x(0)
    c.x(1)
    c.x(2)
    c.x(3)

    for _ in range(depth):
        for i in range(ns-1):
            c.rzz(i, (i + 1) % ns, theta=params[idx+2*i])
            c.iswap(i, (i + 1) % ns, theta=params[idx+2*i+1])
        idx+=2*ns-2

        c.iswap(ns-1, 0, theta=params[idx])
        idx += 1

        for i in range(ns):
            c.rz(i, theta=params[idx+i])
        idx+=ns

    return c, idx

In [ ]:
# circuit visulization, optional
ns_ = ns
p = 1
cirq, idx = U_conventional(ns_, p, np.zeros([1000]))
print("The number of parameters is", idx)
cirq.draw()

In [ ]:
def train(ns, p, e_anal, maxiter=10000, lr=0.01, stddev=1.0):
    _, idx = U_conventional(ns, p, np.zeros([1000]))
    params = tf.Variable(
        initial_value=tf.random.normal(
            shape=[idx], stddev=stddev, dtype=getattr(tf, tc.rdtypestr)
        )
    )

    exp_lr = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=lr, decay_steps=150, decay_rate=0.7
    )
    opt = tf.keras.optimizers.Adam(exp_lr)

    e_list = []
    re_list = []
    params_list = []
    for i in range(maxiter):
        with tf.GradientTape() as tape:
            c, _ = U_conventional(ns, p, params)
            e = tc.templates.measurements.operator_expectation(c, mbd_tf)
        e_list.append(K.real(e).numpy())
        re_list.append(np.abs((e_list[-1] - e_anal) / e_anal))
        params_list.append(params.numpy())
        grads = tape.gradient(e, params)
        opt.apply_gradients(zip([grads], [params]))
        if (i + 1) % 50 == 0 or i==0:
            print("epoch{:>4}, e: {:.10f}, re: {:.10f}".format(i, e_list[-1], re_list[-1]))
    
    params_list.append(params.numpy())
    print(params.numpy())
    
    return e_list, re_list, params_list

In [ ]:
import csv

p = 8
maxiter = 500
lr = 0.1
stddev = np.pi

t = time.gmtime()

# train
with tf.device("/cpu:0"):
    with open('data/h4_square_17_p{}_{}.csv'.format(p, time.strftime("%Y_%m%d_%H_%M_%S",t)), 'w', newline='') as fp1:   # check the filename
        with open('data/h4_square_17_p{}_params_{}.csv'.format(p, time.strftime("%Y_%m%d_%H_%M_%S",t)), 'w', newline='') as fp2:   # check the filename
            writer1 = csv.writer(fp1)
            writer2 = csv.writer(fp2)
            for j in range(50):
                print(j+1)
                e_list, re_list, params_list = train(ns, p, e_anal, maxiter=maxiter, lr=lr, stddev=stddev)
                writer1.writerow([j])
                writer1.writerow(e_list)
                writer1.writerow(re_list)
                writer2.writerow([j])
                for k in range(maxiter+1):
                    writer2.writerow(params_list[k])

## <center> Post-selected VQE

In [ ]:
def U_postselection(n, na, depth, params):
    params = K.cast(params, "complex128")
    c = tc.Circuit(n)
    idx = 0
    
    c.x(1)
    c.x(2)
    c.x(3)
    c.x(4)
    
    for _ in range(depth):
        for i in range(n-1):
            c.rzz(i, (i + 1) % n, theta=params[idx+2*i])
            c.iswap(i, (i + 1) % n, theta=params[idx+2*i+1])
        idx+=2*n-2

        c.iswap(n-1, 0, theta=params[idx])
        idx += 1

        for i in range(n):
            c.rz(i, theta=params[idx+i])
        idx+=n

    return c, idx

In [ ]:
# circuit visulization, optional
ns_ = ns
na = 1
n = ns_ + na
p = 1
cirq, idx = U_postselection(n, na, p, np.zeros([1000]))
print("The number of parameters is", idx)
cirq.draw()

In [ ]:
def postselect(n, na, ns, p, params):
    c, _ = U_postselection(n, na, p, params)

    w = c.wavefunction()[ : 2**ns]
    norm = tf.linalg.norm(w)
    w = w / norm
    
    return tc.Circuit(ns, inputs=w), tf.abs(norm)**2

In [ ]:
# only energy as loss
def train(n, ns, na, p, e_anal, maxiter=10000, lr=0.01, stddev=1.0):
    _, idx = U_postselection(n, na, p, np.zeros([1000]))
    params = tf.Variable(
        initial_value=tf.random.normal(
            shape=[idx], stddev=stddev, dtype=getattr(tf, tc.rdtypestr)
        )
    )

    exp_lr = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=lr, decay_steps=150, decay_rate=0.7
    )
    opt = tf.keras.optimizers.Adam(exp_lr)

    e_list = []
    re_list = []
    params_list = []
    ps_list = []
    for i in range(maxiter):
        with tf.GradientTape() as tape:
            c_s, norm = postselect(n, na, ns, p, params)
            e = tc.templates.measurements.operator_expectation(c_s, mbd_tf)
            ps_list.append(norm.numpy())
        e_list.append(K.real(e).numpy())
        re_list.append(np.abs((e_list[-1] - e_anal) / e_anal))
        params_list.append(params.numpy())
        grads = tape.gradient(e, params)
        opt.apply_gradients(zip([grads], [params]))
        if (i + 1) % 50 == 0 or i==0:
            print("epoch{:>4}, e: {:.10f}, re: {:.10f}".format(i, e_list[-1], re_list[-1]))
    
    params_list.append(params.numpy())
    print(params.numpy())

    return e_list, re_list, params_list, ps_list

In [ ]:
# consider ancilla accuracy in loss
def train(n, ns, na, p, e_anal, maxiter=10000, lr=0.01, stddev=1.0):
    _, idx = U_postselection(n, na, p, np.zeros([1000]))
    params = tf.Variable(
        initial_value=tf.random.normal(
            shape=[idx], stddev=stddev, dtype=getattr(tf, tc.rdtypestr)
        )
    )

    exp_lr = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=lr, decay_steps=150, decay_rate=0.7
    )
    opt = tf.keras.optimizers.Adam(exp_lr)

    e_list = []
    re_list = []
    params_list = []
    ps_list = []
    for i in range(maxiter):
        with tf.GradientTape() as tape:
            c_s, norm = postselect(n, na, ns, p, params)
            e = tc.templates.measurements.operator_expectation(c_s, mbd_tf)
            loss = e - 0.01 * norm
            tape.watch(loss)
    
        e_list.append(K.real(e).numpy())
        re_list.append(np.abs((e_list[-1] - e_anal) / e_anal))
        params_list.append(params.numpy())
        ps_list.append(norm.numpy())
        if (i + 1) % 50 == 0 or i==0:
            print("epoch{:>4}, e: {:.10f}, re: {:.10f}, ps: {:.2f}".format(i, e_list[-1], re_list[-1], ps_list[-1]))
 
        grads = tape.gradient(loss, params)
        opt.apply_gradients(zip([grads], [params]))
        
    params_list.append(params.numpy())
    print(params.numpy())

    return e_list, re_list, params_list, ps_list

In [ ]:
# consider ancilla accuracy with sigmoid function in loss
def train(n, ns, na, p, e_anal, maxiter=10000, lr=0.01, stddev=1.0):
    _, idx = U_postselection(n, na, p, np.zeros([1000]))
    params = tf.Variable(
        initial_value=tf.random.normal(
            shape=[idx], stddev=stddev, dtype=getattr(tf, tc.rdtypestr)
        )
    )

    exp_lr = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=lr, decay_steps=150, decay_rate=0.7
    )
    opt = tf.keras.optimizers.Adam(exp_lr)

    e_list = []
    re_list = []
    params_list = []
    ps_list = []
    for i in range(maxiter):
        with tf.GradientTape() as tape:
            c_s, norm = postselect(n, na, ns, p, params)
            e = tc.templates.measurements.operator_expectation(c_s, mbd_tf)
            loss = e - 0.005 * tf.sigmoid((norm-0.70) * 30)
            tape.watch(loss)
    
        e_list.append(K.real(e).numpy())
        re_list.append(np.abs((e_list[-1] - e_anal) / e_anal))
        params_list.append(params.numpy())
        ps_list.append(norm.numpy())
        if (i + 1) % 50 == 0 or i==0:
            print("epoch{:>4}, e: {:.10f}, re: {:.10f}, ps: {:.2f}, loss: {:.10f}".format(i, e_list[-1], re_list[-1], ps_list[-1], loss))
 
        grads = tape.gradient(loss, params)
        opt.apply_gradients(zip([grads], [params]))
        
    params_list.append(params.numpy())
    print(params.numpy())

    return e_list, re_list, params_list, ps_list

In [ ]:
import csv
na = 1
n = ns + na
p = 8
maxiter = 500
lr = 0.1
stddev = np.pi

t = time.gmtime()

# train
with tf.device("/cpu:0"):
    with open('data/h4_square_17_ps_p{}_sigmoid70_{}.csv'.format(p, time.strftime("%Y_%m%d_%H_%M_%S",t)), 'w', newline='') as fp1:   # check the filename
        with open('data/h4_square_17_ps_p{}_sigmoid70_params_{}.csv'.format(p, time.strftime("%Y_%m%d_%H_%M_%S",t)), 'w', newline='') as fp2:   # check the filename
            writer1 = csv.writer(fp1)
            writer2 = csv.writer(fp2)
            for j in range(50):
                print(j+1)
                e_list, re_list, params_list, ps_list = train(n, ns, na, p, e_anal, maxiter=maxiter, lr=lr, stddev=stddev)
                writer1.writerow([j])
                writer1.writerow(e_list)
                writer1.writerow(re_list)
                writer1.writerow(ps_list)

                writer2.writerow([j])
                writer2.writerows(params_list)